In [ ]:
import numpy as np
import pandas as pd
from dask import dataframe as dd
import os
import plotly.express as px

In [ ]:
update_file = "/home/juuso/Documents/gradu/ADAUSDT_T_DEPTH_2021-10-30/ADAUSDT_T_DEPTH_2021-10-30_depth_update.csv"
snap_file = "/home/juuso/Documents/gradu/ADAUSDT_T_DEPTH_2021-10-30/ADAUSDT_T_DEPTH_2021-10-30_depth_snap.csv"
res_file = "/home/juuso/Documents/gradu/ADAUSDT_T_DEPTH_2021-10-30/parsed.csv"
parsed = "/home/juuso/Documents/gradu/parsed_data/2021-12-22_parsed.csv"
parsed_path = "/home/juuso/Documents/gradu/parsed_data"
hb_logs = "/home/juuso/Documents/hummingbot/local_v2/hummingbot/logs/intensity_sample.csv"
as_logs = "/home/juuso/Documents/gradu/logs/as_env_sample_1673332000.log"

In [ ]:
data_lines = []
with open(as_logs, "r") as f:
    while True:
        line = f.readline()
        if line == "":
            break
        line = line.rstrip().split(":")
        data = line[2].split(",")
        data_lines.append(data)
df = pd.DataFrame(data_lines)
update_df = df[df[0] == "update"]
action_df = df[df[0] == "action"]
action_df.drop(columns=[8,9], inplace=True)

In [ ]:

update_df.columns = ["type", "timestamp", "bid_price","ask_price","mid_price", "vol", "intensity", "inventory", "reservation_price", "spread"]
update_df


In [ ]:
px.line(update_df[::100], x="timestamp", y=["bid_price", "ask_price", "mid_price"])

In [ ]:
px.line(update_df[:100], x="timestamp", y=["mid_price", "reservation_price"])

In [ ]:

action_df.columns = ["type", "timestamp", "action", "price", "amount", "mid_price", 'base',"quote"]
action_df
volume = (action_df.amount.astype(float) * action_df.price.astype(float)).sum()
end_total = (action_df.quote.astype(float) + action_df.base.astype(float) * action_df.mid_price.astype(float)).values[-1]
start_total = (action_df.quote.astype(float) + action_df.base.astype(float) * action_df.mid_price.astype(float)).values[0]

profit = end_total - start_total
break_even_fees = profit / volume
print(f"Break even fees: {break_even_fees}")

In [ ]:
px.line(action_df[::100], x="timestamp", y=["base"])

In [ ]:
for chunk in pd.read_csv(update_file, chunksize=1_000_000):
    ts_chunk = chunk[chunk.timestamp == 1635552439148]
    if ts_chunk.empty:
        continue
    else:
        print('found')
        break

ts_chunk

In [ ]:
df = pd.read_csv("/home/juuso/Documents/gradu/parsed_data/AvellanedaStoikov/data.csv", nrows=1_000_000)
df = df.rename(columns={'size':"trade_size"})
df = df[df.trade_size != 0]

In [ ]:
df['price_diff'] = np.abs(df.price - df.mid_price)
# choose every 10th row
part = df.iloc[::10]

px.scatter(part, x='price_diff', y='trade_size', trendline='ols')
# df

In [ ]:
grouped = df.groupby('price_diff').sum().reset_index()

px.scatter(grouped, x='price_diff', y='trade_size', trendline='ols')